<a href="https://colab.research.google.com/github/FJDaz/maiathon/blob/build_model/Backend/Notebooks/RAG_Spinoza_secours_OPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎓 Spinoza Secours - Notebook Colab

**Modèle :** Mistral 7B + LoRA Fine-tuned  
**Prompt :** Système hybride optimisé (~250-300 tokens)  
**API :** FastAPI + ngrok


## 📦 Cellule 1 : Installation Dépendances


In [1]:
!pip install -q pyngrok fastapi uvicorn transformers peft accelerate bitsandbytes torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 11.4 MB/s eta 0:00:00


## 📚 Cellule 2 : Imports + Configuration

**🔐 Configuration des secrets (à faire une seule fois) :**

1. Clique sur l'icône 🔑 **Secrets** dans le panneau de gauche de Colab
2. Ajoute deux secrets :
   - **`ngrok`** : Ton token ngrok (https://dashboard.ngrok.com/get-started/your-authtoken)
   - **`HuggingFaceToken`** : Ton token Hugging Face (https://huggingface.co/settings/tokens)

Les tokens seront récupérés automatiquement via `userdata.get()`.


In [2]:
import os
import re
import random
import threading
import torch
from typing import Dict, List, Optional
from pyngrok import ngrok
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import uvicorn
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from google.colab import userdata

# Configuration modèle
BASE_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"
ADAPTER_MODEL = "FJDaz/mistral-7b-philosophes-lora"

# Récupération des tokens depuis Colab Secrets
# ⚠️ Configure tes secrets dans Colab : 🔑 Secrets → Ajouter 'ngrok' et 'HuggingFaceToken'
try:
    NGROK_TOKEN = userdata.get('ngrok')
    print("✅ Token ngrok récupéré depuis Colab Secrets")
except:
    print("❌ Token ngrok non trouvé ! Configure le secret 'ngrok' dans Colab Secrets")
    NGROK_TOKEN = None

try:
    HF_TOKEN = userdata.get('HF_TOKEN')
    print("✅ Token Hugging Face récupéré depuis Colab Secrets")
except:
    print("❌ Token Hugging Face non trouvé ! Configure le secret 'HuggingFaceToken' dans Colab Secrets")
    HF_TOKEN = None

# Configuration ngrok
if NGROK_TOKEN:
    ngrok.set_auth_token(NGROK_TOKEN)
else:
    print("⚠️ ngrok ne sera pas configuré sans token")

# Gestion des ports : Port aléatoire pour éviter les conflits
PORT = random.randint(8001, 9000)
print(f"🔌 Port sélectionné : {PORT}")

# Tuer processus sur le port sélectionné si existant
import subprocess
try:
    result = subprocess.run(f"lsof -ti:{PORT}", shell=True, capture_output=True, text=True)
    if result.stdout.strip():
        subprocess.run(f"kill -9 {result.stdout.strip()}", shell=True)
        print(f"🧹 Port {PORT} libéré")
except:
    pass

print("✅ Imports et configuration OK")


✅ Token ngrok récupéré depuis Colab Secrets
✅ Token Hugging Face récupéré depuis Colab Secrets
🔌 Port sélectionné : 8599
✅ Imports et configuration OK


## 🎯 Cellule 3 : Prompt Système Hybride


In [3]:
# Prompt système hybride optimisé (~250 tokens)
SYSTEM_PROMPT_SPINOZA = """Tu ES Spinoza incarné. Tu dialogues avec un élève de Terminale en première personne.

STYLE SPINOZIEN :
- Géométrie des affects : révèle causes nécessaires, déduis
- Dieu = Nature
- Vocabulaire : conatus, affects, puissance d'agir, servitude

SCHÈMES LOGIQUES :
- Identité : Liberté = Connaissance nécessité
- Causalité : Tout a cause nécessaire
- Implication : Joie → augmentation puissance


MÉTHODE :
1. Révèle nécessité causale
2. Distingue servitude (ignorance) vs liberté (connaissance)
3. Exemples concrets modernes

TRANSITIONS (VARIE) :
- "Donc", "mais alors", "Imagine", "Cela implique"
- "Pourtant", "Sauf que", "C'est contradictoire"

RÈGLES :
- Tutoie (tu/ton/ta)
- Concis (2-3 phrases MAX)
- Questionne au lieu d'affirmer
- Tu ES Spinoza, pas un commentateur de Spinoza.
- Langage DÉTENDU : tu peux être un peu familier, c'est OK
- Évite les phrases trop longues ou compliquées

STYLE :
- Langage JEUNE : parle comme un prof cool, pas comme un livre
- Utilise des mots simples, des expressions d'aujourd'hui
- Tu peux dire "ouais", "ben", "genre", "en fait" - c'est naturel
- Réponds DIRECTEMENT à ce que l'élève vient de dire
- Utilise tes idées (causalité, nécessité, liberté = connaissance) mais avec des mots simples"""

INSTRUCTIONS_CONTEXTUELLES = {
    "confusion": "L'élève est confus → Donne UNE analogie concrète simple. Réponds DIRECTEMENT à sa confusion.",
    "resistance": "L'élève résiste → Révèle contradiction avec 'mais alors'. Réponds DIRECTEMENT à son objection.",
    "accord": "L'élève est d'accord → Valide puis AVANCE logiquement avec 'Donc'. Réponds à ce qu'il vient de dire.",
    "neutre": "Élève neutre → Pose question pour faire réfléchir. Réponds à ce qu'il dit."
}

# RAG DÉSACTIVÉ par défaut - trop embrouillant
INSTRUCTION_RAG = """
⚠️ RAG DÉSACTIVÉ - Réponds avec tes propres idées, pas en récitant.
"""

def construire_prompt_complet(contexte: str, use_rag_instruction: bool = True) -> str:
    """
    Construit le prompt complet optimisé

    Args:
        contexte: "accord", "confusion", "resistance", "neutre"
        use_rag_instruction: Si True, ajoute instructions RAG

    Returns:
        Prompt système complet (~250-300 tokens)
    """
    prompt = SYSTEM_PROMPT_SPINOZA

    # Ajouter instruction contextuelle
    if contexte in INSTRUCTIONS_CONTEXTUELLES:
        prompt += f"\n\n{INSTRUCTIONS_CONTEXTUELLES[contexte]}"

    # Ajouter instruction RAG (optionnel)
    if use_rag_instruction:
        prompt += f"\n\n{INSTRUCTION_RAG}"

    return prompt

print("✅ Prompt système hybride chargé")


✅ Prompt système hybride chargé


## 🔍 Cellule 4 : Détection Contexte + Post-Processing


In [4]:
def detecter_contexte(user_input: str) -> str:
    """Détecte le contexte de la réponse utilisateur"""
    text_lower = user_input.lower()

    # Accord
    if any(word in text_lower for word in ['oui', "d'accord", 'exact', 'ok', 'voilà', 'tout à fait']):
        return "accord"

    # Confusion
    if any(phrase in text_lower for phrase in ['comprends pas', 'vois pas', "c'est quoi", 'je sais pas', 'pourquoi', 'rapport']):
        return "confusion"

    # Résistance
    if any(word in text_lower for word in ['mais', 'non', "pas d'accord", 'faux', "n'importe quoi", 'je peux']):
        return "resistance"

    return "neutre"

def nettoyer_reponse(text: str) -> str:
    """Nettoie la réponse générée"""
    text = re.sub(r'\([^)]*[Aa]ttends[^)]*\)', '', text)
    text = re.sub(r'\([^)]*[Pp]oursuis[^)]*\)', '', text)
    text = re.sub(r'\([^)]*[Dd]onne[^)]*\)', '', text)
    text = re.sub(r'[😀-🙏🌀-🗿🚀-🛿]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\s+([.!?])', r'\1', text)
    return text

def limiter_phrases(text: str, max_phrases: int = 4) -> str:
    """Limite le nombre de phrases"""
    phrases = re.split(r'[.!?]+\s+', text)
    phrases = [p.strip() for p in phrases if p.strip()]
    if len(phrases) <= max_phrases:
        return text
    return '. '.join(phrases[:max_phrases]) + '.'

print("✅ Détection contexte + Post-processing OK")


✅ Détection contexte + Post-processing OK


## 🤖 Cellule 5 : Chargement Modèle

**⚠️ Code reproduit exactement depuis app.py - Ne pas modifier**


In [5]:
# =============================================================================
# 🤖 CELLULE : Chargement Modèle Merged (4-BIT)
# =============================================================================

import time

print("=" * 80)
print("🤖 CHARGEMENT MODÈLE MERGED (Spinoza)")
print("=" * 80)
print()

# Configuration modèle
MERGED_DRIVE_PATH = "/content/drive/MyDrive/models/SPSFused"  # Si tu l'as sauvegardé sur Drive
MERGED_HF_REPO = "FJDaz/spinoza-mistral-7b-merged"  # ✅ Ton repo HF

# Déterminer source du modèle
if os.path.exists(MERGED_DRIVE_PATH):
    MODEL_PATH = MERGED_DRIVE_PATH
    MODEL_SOURCE = "Google Drive"
    print(f"📁 Source : Google Drive")
    print(f"   Chemin : {MERGED_DRIVE_PATH}")
else:
    MODEL_PATH = MERGED_HF_REPO
    MODEL_SOURCE = "HuggingFace"
    print(f"📁 Source : HuggingFace")
    print(f"   Repo : {MERGED_HF_REPO}")
    print(f"   ⚠️  Première fois : Téléchargement ~14 GB (20-30 min)")

print()

# Configuration GPU
has_gpu = torch.cuda.is_available()
print(f"🖥️  GPU disponible : {has_gpu}")

if has_gpu:
    # Configuration 4-bit NF4
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )
    device_map = "auto"
    torch_dtype = torch.bfloat16
    print("⚙️  Configuration : 4-bit NF4 quantization")
    print("   VRAM attendue : ~3.84 GB")
else:
    bnb_config = None
    device_map = "cpu"
    torch_dtype = torch.float32
    print("⚠️  Mode CPU (pas de GPU détecté)")

print()

# Vérifier Drive monté
if MODEL_SOURCE == "Google Drive":
    if not os.path.exists("/content/drive"):
        print("📂 Montage Google Drive...")
        from google.colab import drive
        drive.mount('/content/drive')
        print()

# Chargement modèle
print(f"🔄 Chargement modèle depuis {MODEL_SOURCE}...")
print("   (2-5 min depuis Drive, 20-30 min première fois depuis HF)")
print()

start_load = time.time()

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    quantization_config=bnb_config,
    device_map=device_map,
    torch_dtype=torch_dtype,
    token=HF_TOKEN,  # ✅ Utilise HF_TOKEN (pas HuggingFaceToken)
    trust_remote_code=True,
    low_cpu_mem_usage=True
)

load_time = time.time() - start_load
print(f"✅ Modèle chargé en {load_time:.1f}s ({load_time/60:.1f} min)")
print()

# Chargement tokenizer
print("🔄 Chargement tokenizer...")

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    token=HF_TOKEN,  # ✅ Utilise HF_TOKEN
    trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("✅ Tokenizer chargé")
print()

# Vérification VRAM
if has_gpu:
    vram_used = torch.cuda.memory_allocated() / 1024**3
    vram_reserved = torch.cuda.memory_reserved() / 1024**3
    vram_total = torch.cuda.get_device_properties(0).total_memory / 1024**3

    print("📊 VRAM :")
    print(f"   Utilisée : {vram_used:.2f} GB")
    print(f"   Réservée : {vram_reserved:.2f} GB")
    print(f"   Totale   : {vram_total:.2f} GB")
    print()

    if 3.5 < vram_used < 4.5:
        print("   ✅ VRAM conforme aux tests (~3.84 GB)")
    else:
        print(f"   ⚠️  VRAM différente des tests (attendu: ~3.84 GB, actuel: {vram_used:.2f} GB)")

print()
print("=" * 80)
print("✅ MODÈLE MERGED CHARGÉ")
print("=" * 80)
print()
print(f"📋 Configuration :")
print(f"   Modèle      : {MERGED_HF_REPO}")
print(f"   Source      : {MODEL_SOURCE}")
print(f"   Quantization : {'4-bit NF4' if has_gpu else 'FP32 (CPU)'}")
print(f"   Device      : {'GPU (auto)' if has_gpu else 'CPU'}")
if has_gpu:
    print(f"   VRAM        : {vram_used:.2f} GB")
print(f"   Temps chargement : {load_time:.1f}s ({load_time/60:.1f} min)")
print()

print("🎯 Performance attendue (testée) :")
print("   - Tokens/sec  : ~5.0 tok/s (T4 4-bit)")
print("   - Latence     : ~8.10s/question (sans stopping)")
print("   - Latence     : ~5.68s/question (avec stopping) (+30%)")
print()

🤖 CHARGEMENT MODÈLE MERGED (Spinoza)

📁 Source : HuggingFace
   Repo : FJDaz/spinoza-mistral-7b-merged
   ⚠️  Première fois : Téléchargement ~14 GB (20-30 min)

🖥️  GPU disponible : True
⚙️  Configuration : 4-bit NF4 quantization
   VRAM attendue : ~3.84 GB

🔄 Chargement modèle depuis HuggingFace...
   (2-5 min depuis Drive, 20-30 min première fois depuis HF)



config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

✅ Tokenizer chargé

📊 VRAM :
   Utilisée : 3.84 GB
   Réservée : 6.73 GB
   Totale   : 14.74 GB

   ✅ VRAM conforme aux tests (~3.84 GB)

✅ MODÈLE MERGED CHARGÉ

📋 Configuration :
   Modèle      : FJDaz/spinoza-mistral-7b-merged
   Source      : HuggingFace
   Quantization : 4-bit NF4
   Device      : GPU (auto)
   VRAM        : 3.84 GB
   Temps chargement : 241.7s (4.0 min)

🎯 Performance attendue (testée) :
   - Tokens/sec  : ~5.0 tok/s (T4 4-bit)
   - Latence     : ~8.10s/question (sans stopping)
   - Latence     : ~5.68s/question (avec stopping) (+30%)



## 💬 Cellule 6 : Fonction spinoza_repond()

**Utilise le prompt système hybride optimisé**


In [6]:
# Historique conversation (format: [user, assistant])
conversation_history = []

def spinoza_repond(message: str) -> str:
    """
    Génère une réponse de Spinoza avec prompt hybride adaptatif

    Args:
        message: Message de l'utilisateur

    Returns:
        Réponse de Spinoza nettoyée
    """
    global conversation_history

    # Détecter contexte
    contexte = detecter_contexte(message)

    # Construire prompt adaptatif
    system_prompt = construire_prompt_complet(contexte, use_rag_instruction=False)

    # Formatage Mistral style
    prompt_parts = [f"<s>[INST] {system_prompt}\n\n"]

    # Ajouter historique (4 derniers échanges max)
    for entry in conversation_history[-4:]:
        prompt_parts.append(f"{entry[0]} [/INST] {entry[1]}</s>[INST] ")

    prompt_parts.append(f"{message} [/INST]")
    text = "".join(prompt_parts)

    # Tokenization
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    input_length = inputs['input_ids'].shape[1]

    # Génération
    device_type = "cuda" if torch.cuda.is_available() else "cpu"
    dtype = torch.bfloat16 if device_type == "cuda" else torch.float32

    with torch.autocast(device_type=device_type, dtype=dtype):
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    # Décodage
    new_tokens = outputs[0][input_length:]
    response = tokenizer.decode(new_tokens, skip_special_tokens=True)

    # Post-processing
    response = nettoyer_reponse(response)
    response = limiter_phrases(response, max_phrases=3)

    # Mettre à jour historique
    conversation_history.append([message, response])

    return response

print("✅ Fonction spinoza_repond() créée")


✅ Fonction spinoza_repond() créée


## STOPPING CRITERIA

In [7]:
# =============================================================================
# 🛑 CELLULE : STOPPING CRITERIA + ÉVALUATION INCRÉMENTALE
# =============================================================================
# À ajouter APRÈS cellule spinoza_repond() et AVANT cellule API FastAPI
# =============================================================================

import json
from transformers import StoppingCriteria
from pydantic import BaseModel
from typing import Optional

# =============================================================================
# STOPPING CRITERIA - Version Standard (Meilleure Performance)
# =============================================================================

class StopAfterCompleteSentences(StoppingCriteria):
    """
    Arrête après N phrases complètes avec ponctuation finale
    - Compte vraiment les phrases (pas estimation)
    - Détecte ponctuation FINALE (pas toute ponctuation)
    - Décode tous les 5 tokens (performance optimisée)

    Testé : +30% gain latency (8.10s → 5.68s) avec 100% réponses complètes
    """
    def __init__(self, tokenizer, input_length, max_sentences=2, min_tokens=20):
        self.tokenizer = tokenizer
        self.input_length = input_length
        self.max_sentences = max_sentences
        self.min_tokens = min_tokens
        self.sentence_count = 0
        self.last_checked_length = 0
        self.decode_interval = 5  # Décode tous les 5 tokens

    def __call__(self, input_ids, scores, **kwargs):
        # Vérifier minimum de tokens générés
        generated_length = input_ids.shape[1] - self.input_length
        if generated_length < self.min_tokens:
            return False

        # Décodage conditionnel (tous les decode_interval tokens)
        if generated_length - self.last_checked_length < self.decode_interval:
            return False

        self.last_checked_length = generated_length

        # Décoder seulement les nouveaux tokens
        generated_ids = input_ids[0][self.input_length:]
        text = self.tokenizer.decode(generated_ids, skip_special_tokens=True)

        # Compter les phrases COMPLÈTES (finissant par ponctuation)
        import re
        sentence_endings = re.findall(r'[.!?]\s*$', text)
        self.sentence_count = len(sentence_endings)

        # Arrêter si nombre de phrases atteint
        return self.sentence_count >= self.max_sentences


# =============================================================================
# FONCTION SPINOZA AVEC STOPPING CRITERIA (v2)
# =============================================================================

def spinoza_repond_avec_stopping(message: str, use_stopping_criteria: bool = True) -> str:
    """
    Génère une réponse de Spinoza avec stopping criteria optimisé

    Args:
        message: Message de l'utilisateur
        use_stopping_criteria: Si True, utilise stopping criteria (défaut: True)

    Returns:
        Réponse de Spinoza nettoyée
    """
    global conversation_history

    # Détecter contexte
    contexte = detecter_contexte(message)

    # Construire prompt adaptatif
    system_prompt = construire_prompt_complet(contexte, use_rag_instruction=False)

    # Formatage Mistral style
    prompt_parts = [f"<s>[INST] {system_prompt}\n\n"]

    # Ajouter historique (4 derniers échanges max)
    for entry in conversation_history[-4:]:
        prompt_parts.append(f"{entry[0]} [/INST] {entry[1]}</s>[INST] ")

    # FORMAT CORRIGÉ : Indiquer explicitement que Spinoza doit parler
    prompt_parts.append(f"Élève : {message}\n\nSpinoza : [/INST]")  # ✅ FIX
    text = "".join(prompt_parts)

    # Tokenization
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    input_length = inputs['input_ids'].shape[1]

    # Configuration stopping criteria
    stopping_criteria_list = None
    if use_stopping_criteria:
        stopping_criteria_list = [
            StopAfterCompleteSentences(
                tokenizer=tokenizer,
                input_length=input_length,
                max_sentences=2,  # 2 phrases max (testé optimal)
                min_tokens=20     # Minimum 20 tokens
            )
        ]

    # Génération
    device_type = "cuda" if torch.cuda.is_available() else "cpu"
    dtype = torch.bfloat16 if device_type == "cuda" else torch.float32

    with torch.autocast(device_type=device_type, dtype=dtype):
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            stopping_criteria=stopping_criteria_list if use_stopping_criteria else None
        )

    # Décodage
    new_tokens = outputs[0][input_length:]
    response = tokenizer.decode(new_tokens, skip_special_tokens=True)

    # Post-processing
    response = nettoyer_reponse(response)
    response = limiter_phrases(response, max_phrases=3)

    # Mettre à jour historique
    conversation_history.append([message, response])

    return response


# =============================================================================
# MODÈLES PYDANTIC POUR ÉVALUATION
# =============================================================================

class EvaluateRequest(BaseModel):
    dialogue: str
    score_front: Optional[int] = 0  # Score du frontend (optionnel)


# =============================================================================
# ÉVALUATION INCRÉMENTALE (repris de votre cellule 18 corrigée)
# =============================================================================

# Prompt évaluation incrémentale (court, rapide)
PROMPT_EVALUATION_INCREMENTAL = """Évalue rapidement ces 2 derniers échanges (0-10) :

{dialogue_recent}

CRITÈRES :
• Compréhension : L'élève comprend-il les idées ? (0=rien compris, 10=parfait)
• Coopération : Coopère-t-il activement ? (0=refuse, 10=très engagé)
• Progression : Sa pensée progresse-t-elle ? (0=aucune, 10=grande évolution)

EXEMPLES :
- "J'en ai rien à faire" → 1, 1, 0
- "Je comprends pas" → 3, 5, 2
- "Ah oui ! c'est plus précis !" → 7, 8, 7
- "Je comprends maintenant !" → 9, 10, 9

Réponds UNIQUEMENT avec ce JSON :

{{
 "comprehension": X,
 "cooperation": Y,
 "progression": Z,
 "total": X+Y+Z
}}"""

# Stockage scores incrémentaux
incremental_scores = {}


def evaluer_incremental(dialogue: str, debug: bool = False, return_raw: bool = False):
    """
    Évaluation légère au fil de l'eau (tous les 2 échanges)

    Args:
        dialogue: Le dialogue à évaluer
        debug: Si True, affiche la réponse brute du modèle
        return_raw: Si True, retourne aussi la réponse brute

    Returns:
        dict ou tuple: Scores évalués (+ raw_response si return_raw=True)
    """
    # Extraire les 2 derniers échanges
    lines = [l.strip() for l in dialogue.split('\n') if l.strip()]
    if len(lines) > 4:
        recent_exchanges = '\n'.join(lines[-4:])
    else:
        recent_exchanges = dialogue

    prompt_eval = PROMPT_EVALUATION_INCREMENTAL.format(dialogue_recent=recent_exchanges)
    prompt_eval_formatted = f"<s>[INST] {prompt_eval} [/INST]"

    inputs = tokenizer(prompt_eval_formatted, return_tensors="pt").to(model.device)
    input_length = inputs['input_ids'].shape[1]

    device_type = "cuda" if torch.cuda.is_available() else "cpu"
    dtype = torch.bfloat16 if device_type == "cuda" else torch.float32

    # Inférence rapide
    with torch.autocast(device_type=device_type, dtype=dtype):
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.1,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    new_tokens = outputs[0][input_length:]
    reponse_eval = tokenizer.decode(new_tokens, skip_special_tokens=True)

    if debug:
        print(f"🔍 [DEBUG] Réponse brute: {reponse_eval[:500]}")

    # Parser JSON (3 stratégies de fallback)
    details_model = None

    # Stratégie 1: Pattern spécifique avec tous les champs
    json_pattern = r'\{[^{}]*"(?:comprehension|compréhension)"[^{}]*"(?:cooperation|coopération)"[^{}]*"progression"[^{}]*"total"[^{}]*\}'
    json_match = re.search(json_pattern, reponse_eval, re.DOTALL)

    if json_match:
        try:
            details_model = json.loads(json_match.group(0).strip())
        except json.JSONDecodeError as e:
            if debug:
                print(f"⚠️ Erreur parsing JSON (stratégie 1): {e}")

    # Stratégie 2: Chercher premier bloc JSON valide
    if not details_model:
        json_pattern_fallback = r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}'
        json_match = re.search(json_pattern_fallback, reponse_eval, re.DOTALL)
        if json_match:
            try:
                details_model = json.loads(json_match.group(0).strip())
            except json.JSONDecodeError:
                pass

    # Stratégie 3: Parser toute la réponse
    if not details_model:
        try:
            details_model = json.loads(reponse_eval.strip())
        except json.JSONDecodeError:
            if debug:
                print(f"⚠️ Impossible de parser: {reponse_eval[:300]}")

    # Normaliser les clés (gérer accents)
    if details_model and isinstance(details_model, dict):
        import unicodedata

        def normalize_key(key: str) -> str:
            key_normalized = unicodedata.normalize('NFD', key.lower())
            key_normalized = ''.join(c for c in key_normalized if unicodedata.category(c) != 'Mn')

            mapping = {
                "comprehension": "comprehension",
                "compréhension": "comprehension",
                "cooperation": "cooperation",
                "coopération": "cooperation",
                "progression": "progression",
                "total": "total"
            }
            return mapping.get(key_normalized, key_normalized)

        normalized_model = {}
        for key, value in details_model.items():
            normalized_key = normalize_key(key)
            if normalized_key not in normalized_model:
                normalized_model[normalized_key] = value

        details_model = normalized_model

        # Valider champs
        required_fields = ["comprehension", "cooperation", "progression", "total"]
        for field in required_fields:
            if field not in details_model:
                details_model[field] = 5 if field != "total" else 15

        # Recalculer total si nécessaire
        if not isinstance(details_model["total"], (int, float)):
            details_model["total"] = sum(details_model.get(f, 5) for f in ["comprehension", "cooperation", "progression"])
    else:
        # Valeurs par défaut
        print(f"⚠️ JSON non trouvé, utilisation valeurs par défaut")
        details_model = {"comprehension": 5, "cooperation": 5, "progression": 5, "total": 15}

    if return_raw:
        return details_model, reponse_eval
    return details_model


# =============================================================================
# ENDPOINT API POUR ÉVALUATION
# =============================================================================
# Ces endpoints seront ajoutés à votre API FastAPI dans la cellule suivante

print("✅ Stopping criteria + Évaluation incrémentale chargés")
print()
print("📋 Nouvelles fonctions disponibles :")
print("   - spinoza_repond_avec_stopping(message, use_stopping_criteria=True)")
print("   - evaluer_incremental(dialogue, debug=False, return_raw=False)")
print()
print("🎯 Performance attendue avec stopping criteria :")
print("   - Baseline sans stopping : ~8.10s/question")
print("   - Avec stopping (Standard) : ~5.68s/question (+30% gain)")
print()
print("🔧 Prochaine étape : Ajouter les endpoints API dans la cellule FastAPI")


✅ Stopping criteria + Évaluation incrémentale chargés

📋 Nouvelles fonctions disponibles :
   - spinoza_repond_avec_stopping(message, use_stopping_criteria=True)
   - evaluer_incremental(dialogue, debug=False, return_raw=False)

🎯 Performance attendue avec stopping criteria :
   - Baseline sans stopping : ~8.10s/question
   - Avec stopping (Standard) : ~5.68s/question (+30% gain)

🔧 Prochaine étape : Ajouter les endpoints API dans la cellule FastAPI


## 🌐 Cellule 7 : API FastAPI + ngrok


In [8]:
# Questions d'amorce
QUESTIONS_BAC = [
    "La liberté est-elle une illusion ?",
    "Suis-je esclave de mes désirs ?",
    "Puis-je maîtriser mes émotions ?",
    "La joie procure-t-elle un pouvoir ?",
    "Peut-on désirer sans souffrir ?",
]

# Modèles Pydantic
class ChatRequest(BaseModel):
    message: str
    history: Optional[List[List[str]]] = None

class ChatResponse(BaseModel):
    reply: str
    history: List[List[str]]

# API FastAPI
app = FastAPI()

# CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # ⚠️ À restreindre en production
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/")
def root():
    """Endpoint racine - Informations sur l'API"""
    return {
        "name": "Spinoza Secours API",
        "model": "Mistral 7B + LoRA",
        "status": "running",
        "endpoints": {
            "health": "GET /health",
            "init": "GET /init",
            "chat": "POST /chat"
        }
    }

@app.get("/health")
def health():
    return {"status": "ok", "model": "Mistral 7B + LoRA"}

@app.get("/init")
def init():
    global conversation_history
    conversation_history = []
    question = random.choice(QUESTIONS_BAC)
    greeting = f"Bonjour ! Je suis Spinoza. Discutons :\n\n**{question}**\n\nQu'en penses-tu ?"
    return {
        "greeting": greeting,
        "history": [[None, greeting]]
    }

@app.post("/chat")
def chat(req: ChatRequest):
    global conversation_history

    # Mettre à jour historique si fourni
    if req.history:
        conversation_history = req.history

    # Générer réponse
    reply = spinoza_repond(req.message)

    return {
        "reply": reply,
        "history": conversation_history
    }

print("✅ API FastAPI créée")


✅ API FastAPI créée


In [9]:
# =============================================================================
# 🌐 CELLULE : ENDPOINTS API POUR STOPPING CRITERIA + ÉVALUATION
# =============================================================================
# À ajouter APRÈS la cellule API FastAPI (après @app.post("/chat"))
# =============================================================================

from fastapi import Query

# =============================================================================
# ENDPOINT : Chat avec stopping criteria
# =============================================================================

@app.post("/chat/optimized")
def chat_optimized(req: ChatRequest):
    """
    Endpoint de chat AVEC stopping criteria optimisé
    - Utilise StopAfterCompleteSentences (testé : +30% gain)
    - Limite génération à 2 phrases complètes
    - Latence réduite : ~5.68s au lieu de ~8.10s

    Usage depuis Frontend :
        POST /chat/optimized
        Body: {"message": "...", "history": [...]}
    """
    global conversation_history

    # Mettre à jour historique si fourni
    if req.history:
        conversation_history = req.history

    # Générer réponse AVEC stopping criteria
    reply = spinoza_repond_avec_stopping(req.message, use_stopping_criteria=True)

    return {
        "reply": reply,
        "history": conversation_history,
        "optimized": True  # Flag pour indiquer que stopping criteria est actif
    }


# =============================================================================
# ENDPOINT : Évaluation incrémentale
# =============================================================================

@app.post("/evaluate/incremental")
def evaluate_incremental(
    req: EvaluateRequest,
    debug: bool = Query(False, description="Activer le mode debug pour voir la réponse brute du modèle")
):
    """
    Évaluation légère au fil de l'eau (tous les 2 échanges)
    - Appelé par le frontend après chaque 2 échanges
    - Stocke les scores pour l'évaluation finale
    - Retourne seulement les scores (pas de message final)

    Paramètre debug (query string):
        ?debug=true - Retourne aussi la réponse brute du modèle
        Exemple: POST /evaluate/incremental?debug=true

    Usage depuis Frontend :
        POST /evaluate/incremental
        Body: {"dialogue": "Élève: ...\nSpinoza: ...\n..."}
    """
    # Évaluer le dialogue récent
    if debug:
        details_model, raw_response = evaluer_incremental(req.dialogue, debug=True, return_raw=True)
    else:
        details_model = evaluer_incremental(req.dialogue, debug=False, return_raw=False)
        raw_response = None

    # Stocker pour l'évaluation finale
    dialogue_id = hash(req.dialogue)

    if dialogue_id not in incremental_scores:
        incremental_scores[dialogue_id] = []

    incremental_scores[dialogue_id].append({
        "scores": details_model,
        "exchange_count": len(incremental_scores[dialogue_id]) + 1
    })

    # Préparer la réponse
    response = {
        "scores": details_model,
        "exchange_count": len(incremental_scores[dialogue_id]),
        "accumulated": len(incremental_scores[dialogue_id]) > 0
    }

    # Ajouter la réponse brute si debug demandé
    if debug and raw_response:
        response["debug"] = {
            "raw_model_response": raw_response[:500],
            "parsing_success": details_model.get("total", 0) != 15
        }

    return response


# =============================================================================
# ENDPOINT : Comparaison A/B (avec et sans stopping criteria)
# =============================================================================

@app.post("/chat/compare")
def chat_compare(req: ChatRequest):
    """
    Endpoint de test pour comparer les 2 versions
    - Génère une réponse SANS stopping criteria (baseline)
    - Génère une réponse AVEC stopping criteria (optimized)
    - Retourne les deux avec latences respectives

    ⚠️ ATTENTION : Génère 2 réponses → 2x plus lent
    Usage : Tests uniquement, pas en production

    Retourne :
        {
            "baseline": {"reply": "...", "latency": X.XXs},
            "optimized": {"reply": "...", "latency": X.XXs},
            "improvement": "+XX%"
        }
    """
    import time
    global conversation_history

    # Mettre à jour historique
    if req.history:
        conversation_history = req.history

    # Version 1 : SANS stopping criteria
    start_baseline = time.time()
    reply_baseline = spinoza_repond_avec_stopping(req.message, use_stopping_criteria=False)
    latency_baseline = time.time() - start_baseline

    # Version 2 : AVEC stopping criteria
    # Restaurer l'historique (car spinoza_repond_avec_stopping l'a modifié)
    conversation_history = req.history if req.history else []

    start_optimized = time.time()
    reply_optimized = spinoza_repond_avec_stopping(req.message, use_stopping_criteria=True)
    latency_optimized = time.time() - start_optimized

    # Calculer amélioration
    if latency_baseline > 0:
        improvement_pct = ((latency_baseline - latency_optimized) / latency_baseline) * 100
    else:
        improvement_pct = 0

    return {
        "baseline": {
            "reply": reply_baseline,
            "latency": f"{latency_baseline:.2f}s"
        },
        "optimized": {
            "reply": reply_optimized,
            "latency": f"{latency_optimized:.2f}s"
        },
        "improvement": f"+{improvement_pct:.1f}%" if improvement_pct > 0 else f"{improvement_pct:.1f}%",
        "history": conversation_history
    }


# =============================================================================
# ENDPOINT : Récupérer les scores incrémentaux
# =============================================================================

@app.get("/evaluate/incremental/scores")
def get_incremental_scores():
    """
    Récupère tous les scores incrémentaux stockés

    Retourne :
        {
            "dialogue_count": X,
            "scores": [
                {"dialogue_id": ..., "exchanges": [...]},
                ...
            ]
        }
    """
    scores_list = []
    for dialogue_id, exchanges in incremental_scores.items():
        scores_list.append({
            "dialogue_id": dialogue_id,
            "exchanges": exchanges
        })

    return {
        "dialogue_count": len(incremental_scores),
        "scores": scores_list
    }


print("✅ Endpoints API ajoutés :")
print()
print("📡 Nouveaux endpoints disponibles :")
print("   POST /chat/optimized          - Chat avec stopping criteria (+30% gain)")
print("   POST /evaluate/incremental    - Évaluation au fil de l'eau")
print("   POST /chat/compare            - Comparaison A/B (tests uniquement)")
print("   GET  /evaluate/incremental/scores - Récupérer scores stockés")
print()
print("🎯 Recommandation production :")
print("   - Utiliser /chat/optimized au lieu de /chat")
print("   - Appeler /evaluate/incremental tous les 2 échanges")
print()
print("🔧 Test rapide :")
print('   curl -X POST "http://localhost:{PORT}/chat/optimized" \\')
print('        -H "Content-Type: application/json" \\')
print('        -d \'{"message": "La liberté est-elle une illusion ?"}\'')


✅ Endpoints API ajoutés :

📡 Nouveaux endpoints disponibles :
   POST /chat/optimized          - Chat avec stopping criteria (+30% gain)
   POST /evaluate/incremental    - Évaluation au fil de l'eau
   POST /chat/compare            - Comparaison A/B (tests uniquement)
   GET  /evaluate/incremental/scores - Récupérer scores stockés

🎯 Recommandation production :
   - Utiliser /chat/optimized au lieu de /chat
   - Appeler /evaluate/incremental tous les 2 échanges

🔧 Test rapide :
   curl -X POST "http://localhost:{PORT}/chat/optimized" \
        -H "Content-Type: application/json" \
        -d '{"message": "La liberté est-elle une illusion ?"}'


In [15]:
# =============================================================================
# 🎯 ENDPOINT : /evaluate - ÉVALUATION FINALE (CONFORME AU RÉCAP)
# =============================================================================
# Basé sur : RECAP_LOGIQUE_EVALUATION_MAIATHON.md
# =============================================================================

import json

# =============================================================================
# PROMPT MESSAGE FINAL (Version Standard)
# =============================================================================

PROMPT_MESSAGE_FINAL = """Tu es Spinoza.

⚠️ RÈGLE ABSOLUE : RÉPONDS UNIQUEMENT EN FRANÇAIS. Aucun mot en anglais, aucune traduction. Tout le message doit être en français.

En t'inspirant EXCLUSIVEMENT de ton propre système philosophique (Éthique, conatus, affects, puissance d'agir, servitude vs liberté, Dieu = Nature),

rédige un message bref à l'élève.

Structure (obligatoire) :
1. Un compliment sincère lié à son niveau global.
2. Un conseil précis basé sur son critère le plus faible.
3. Un surnom symbolique et positif, tiré de ton univers conceptuel (ex: "puissance d'agir", "essence active", "affect joyeux").

Maximum 3 phrases.
Style concis, poétique, jamais condescendant.

Dialogue complet :
{dialogue}

Scores de l'élève :
- Compréhension : {comprehension}/10
- Coopération : {cooperation}/10
- Progression : {progression}/10

Critère le plus faible : {critere_faible}"""


# =============================================================================
# FONCTIONS UTILITAIRES
# =============================================================================

def agreger_scores_incrementaux(dialogue_id: int):
    """
    Agrège les scores incrémentaux avec moyenne pondérée
    Le dernier échange est plus important (poids croissant)

    Returns:
        dict: Scores agrégés ou None si pas de scores
    """
    if dialogue_id not in incremental_scores or len(incremental_scores[dialogue_id]) == 0:
        return None

    scores_list = incremental_scores[dialogue_id]
    n = len(scores_list)

    # Poids croissant : [1, 2, 3, ...]
    weights = list(range(1, n + 1))
    total_weight = sum(weights)

    # Moyenne pondérée
    comprehension_weighted = sum(s["scores"]["comprehension"] * weights[i] for i, s in enumerate(scores_list)) / total_weight
    cooperation_weighted = sum(s["scores"]["cooperation"] * weights[i] for i, s in enumerate(scores_list)) / total_weight
    progression_weighted = sum(s["scores"]["progression"] * weights[i] for i, s in enumerate(scores_list)) / total_weight

    total = comprehension_weighted + cooperation_weighted + progression_weighted

    return {
        "comprehension": round(comprehension_weighted, 1),
        "cooperation": round(cooperation_weighted, 1),
        "progression": round(progression_weighted, 1),
        "total": round(total, 1)
    }


def generer_message_final(dialogue: str, details_model: dict) -> str:
    """
    Génère le message final personnalisé de Spinoza

    Args:
        dialogue: Dialogue complet
        details_model: Scores du modèle {comprehension, cooperation, progression, total}

    Returns:
        str: Message final de Spinoza
    """
    # Trouver le critère le plus faible
    criteres = {
        "comprehension": details_model.get("comprehension", 5),
        "cooperation": details_model.get("cooperation", 5),
        "progression": details_model.get("progression", 5)
    }
    critere_faible = min(criteres, key=criteres.get)

    # Construire le prompt
    prompt_message = PROMPT_MESSAGE_FINAL.format(
        dialogue=dialogue,
        comprehension=details_model.get("comprehension", 5),
        cooperation=details_model.get("cooperation", 5),
        progression=details_model.get("progression", 5),
        critere_faible=critere_faible
    )

    # Formatage Mistral
    prompt_formatted = f"<s>[INST] {prompt_message} [/INST]"

    inputs = tokenizer(prompt_formatted, return_tensors="pt").to(model.device)
    input_length = inputs['input_ids'].shape[1]

    device_type = "cuda" if torch.cuda.is_available() else "cpu"
    dtype = torch.bfloat16 if device_type == "cuda" else torch.float32

    # Inférence avec température haute pour créativité
    with torch.autocast(device_type=device_type, dtype=dtype):
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=1.1,  # Haute pour créativité
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    new_tokens = outputs[0][input_length:]
    message_final = tokenizer.decode(new_tokens, skip_special_tokens=True)

    # Nettoyer le message
    message_final = nettoyer_reponse(message_final)

    return message_final


def evaluer_dialogue_complet(dialogue: str, debug: bool = False, return_raw: bool = False):
    """
    Évaluation COMPLÈTE du dialogue (utilise PROMPT_EVALUATION, pas INCREMENTAL)
    - Évalue TOUT le dialogue (pas seulement 2 derniers échanges)
    - Utilise grille détaillée avec PRINCIPE BIENVEILLANT
    - Pour évaluation finale Maïeuthon

    Args:
        dialogue: Le dialogue COMPLET à évaluer
        debug: Si True, affiche la réponse brute du modèle
        return_raw: Si True, retourne aussi la réponse brute

    Returns:
        dict ou tuple: Scores évalués (+ raw_response si return_raw=True)
    """
    # Utiliser PROMPT_EVALUATION (défini dans cellule 17 du notebook)
    # Ce prompt a le PRINCIPE BIENVEILLANT et évalue TOUT le dialogue
    prompt_eval = PROMPT_EVALUATION.format(dialogue=dialogue)

    # Formatage Mistral
    prompt_eval_formatted = f"<s>[INST] {prompt_eval} [/INST]"

    inputs = tokenizer(prompt_eval_formatted, return_tensors="pt").to(model.device)
    input_length = inputs['input_ids'].shape[1]

    device_type = "cuda" if torch.cuda.is_available() else "cpu"
    dtype = torch.bfloat16 if device_type == "cuda" else torch.float32

    # Inférence (température basse pour JSON stable)
    with torch.autocast(device_type=device_type, dtype=dtype):
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,  # Plus long que incremental (dialogue complet)
            temperature=0.1,     # Strict pour JSON
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    new_tokens = outputs[0][input_length:]
    reponse_eval = tokenizer.decode(new_tokens, skip_special_tokens=True)

    if debug:
        print(f"🔍 [DEBUG EVAL COMPLÈTE] Réponse brute: {reponse_eval[:500]}")

    # Parser JSON (même logique que evaluer_incremental)
    details_model = None

    # Stratégie 1: Pattern spécifique
    json_pattern = r'\{[^{}]*"(?:comprehension|compréhension)"[^{}]*"(?:cooperation|coopération)"[^{}]*"progression"[^{}]*"total"[^{}]*\}'
    json_match = re.search(json_pattern, reponse_eval, re.DOTALL)

    if json_match:
        try:
            details_model = json.loads(json_match.group(0).strip())
        except json.JSONDecodeError as e:
            if debug:
                print(f"⚠️ Erreur parsing JSON (stratégie 1): {e}")

    # Stratégie 2: Chercher premier bloc JSON valide
    if not details_model:
        json_pattern_fallback = r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}'
        json_match = re.search(json_pattern_fallback, reponse_eval, re.DOTALL)
        if json_match:
            try:
                details_model = json.loads(json_match.group(0).strip())
            except json.JSONDecodeError:
                pass

    # Stratégie 3: Parser toute la réponse
    if not details_model:
        try:
            details_model = json.loads(reponse_eval.strip())
        except json.JSONDecodeError:
            if debug:
                print(f"⚠️ Impossible de parser: {reponse_eval[:300]}")

    # Normaliser les clés (même logique que incremental)
    if details_model and isinstance(details_model, dict):
        import unicodedata

        def normalize_key(key: str) -> str:
            key_normalized = unicodedata.normalize('NFD', key.lower())
            key_normalized = ''.join(c for c in key_normalized if unicodedata.category(c) != 'Mn')

            mapping = {
                "comprehension": "comprehension",
                "compréhension": "comprehension",
                "cooperation": "cooperation",
                "coopération": "cooperation",
                "progression": "progression",
                "total": "total"
            }
            return mapping.get(key_normalized, key_normalized)

        normalized_model = {}
        for key, value in details_model.items():
            normalized_key = normalize_key(key)
            if normalized_key not in normalized_model:
                normalized_model[normalized_key] = value

        details_model = normalized_model

        # Valider champs
        required_fields = ["comprehension", "cooperation", "progression", "total"]
        for field in required_fields:
            if field not in details_model:
                details_model[field] = 5 if field != "total" else 15

        # Recalculer total si nécessaire
        if not isinstance(details_model["total"], (int, float)):
            details_model["total"] = sum(details_model.get(f, 5) for f in ["comprehension", "cooperation", "progression"])
    else:
        # Valeurs par défaut
        print(f"⚠️ JSON non trouvé, utilisation valeurs par défaut")
        details_model = {"comprehension": 5, "cooperation": 5, "progression": 5, "total": 15}

    if return_raw:
        return details_model, reponse_eval
    return details_model


# =============================================================================
# ENDPOINT /evaluate - Évaluation finale (CONFORME)
# =============================================================================

@app.post("/evaluate")
def evaluate_final(req: EvaluateRequest):
    """
    Évaluation finale du dialogue COMPLET (conforme au RECAP_LOGIQUE_EVALUATION_MAIATHON.md)

    Processus :
    1. Vérifier si scores incrémentaux disponibles
       - Si OUI : Agréger scores incrémentaux (moyenne pondérée)
       - Si NON : Évaluation complète du dialogue
    2. Générer message final personnalisé (température haute, créatif)
    3. Calculer score final = score_front + score_backend (SOMME, pas pondération)

    Paramètres :
        dialogue: Dialogue complet (string)
        score_front: Score frontend final (int)

    Retourne :
        {
            "score_final": X,  # score_front + score_backend
            "message_final": "Message personnalisé de Spinoza...",
            "details_model": {
                "comprehension": X,
                "cooperation": Y,
                "progression": Z,
                "total": X+Y+Z
            }
        }
    """
    dialogue_id = hash(req.dialogue)
    score_front = req.score_front if hasattr(req, 'score_front') else 0

    print(f"📊 [EVALUATE] Début évaluation finale")
    print(f"   Score frontend : {score_front}")
    print(f"   Dialogue ID : {dialogue_id}")

    # =============================================================================
    # ÉTAPE 1 : Obtenir scores backend
    # =============================================================================

    # Option 1 : Scores incrémentaux disponibles (OPTIMISATION)
    details_model = agreger_scores_incrementaux(dialogue_id)

    if details_model:
        print(f"   ✅ Scores incrémentaux agrégés (optimisation)")
        print(f"      Compréhension : {details_model['comprehension']}")
        print(f"      Coopération : {details_model['cooperation']}")
        print(f"      Progression : {details_model['progression']}")
    else:
        # Option 2 : Fallback - Évaluation complète
        print(f"   ⚠️  Pas de scores incrémentaux → Évaluation complète")
        details_model = evaluer_dialogue_complet(req.dialogue, debug=False, return_raw=False)

    score_backend = details_model.get("total", 15)

    # =============================================================================
    # ÉTAPE 2 : Générer message final personnalisé
    # =============================================================================

    print(f"   🔄 Génération message final...")
    message_final = generer_message_final(req.dialogue, details_model)
    print(f"   ✅ Message généré ({len(message_final)} caractères)")

    # =============================================================================
    # ÉTAPE 3 : Calculer score final (SOMME)
    # =============================================================================

    score_final = score_front + score_backend  # ⚠️ SOMME, pas pondération !

    print(f"📊 [EVALUATE] Résultat final")
    print(f"   Score backend : {score_backend}")
    print(f"   Score frontend : {score_front}")
    print(f"   Score final : {score_final}")
    print()

    return {
        "score_final": score_final,
        "message_final": message_final,
        "details_model": details_model
    }


print("✅ Endpoint /evaluate ajouté (CONFORME AU RÉCAP)")
print()
print("📡 Endpoint disponible :")
print("   POST /evaluate - Évaluation finale conforme")
print()
print("🎯 Conformité :")
print("   - Score final = score_front + score_backend (SOMME)")
print("   - Optimisation : Agrégation scores incrémentaux si disponibles")
print("   - Message final personnalisé généré par le modèle")
print("   - Température haute (1.1) pour créativité du message")
print()
print("⚠️  IMPORTANT : PROMPT_EVALUATION doit être défini (cellule 17)")


✅ Endpoint /evaluate ajouté (CONFORME AU RÉCAP)

📡 Endpoint disponible :
   POST /evaluate - Évaluation finale conforme

🎯 Conformité :
   - Score final = score_front + score_backend (SOMME)
   - Optimisation : Agrégation scores incrémentaux si disponibles
   - Message final personnalisé généré par le modèle
   - Température haute (1.1) pour créativité du message

⚠️  IMPORTANT : PROMPT_EVALUATION doit être défini (cellule 17)


## 🚀 Cellule 8 : Lancement Serveur + ngrok


In [16]:
# Lancer FastAPI en background avec port dynamique
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=PORT, log_level="error")

thread = threading.Thread(target=run_server, daemon=True)
thread.start()

# Attendre que le serveur démarre
import time
time.sleep(3)

# Tunnel ngrok sur le port dynamique
tunnel = ngrok.connect(PORT)
# Extraire l'URL publique depuis l'objet NgrokTunnel
# L'objet NgrokTunnel a une méthode public_url ou on peut extraire depuis str()
if hasattr(tunnel, 'public_url'):
    public_url = tunnel.public_url
elif hasattr(tunnel, 'data') and 'public_url' in tunnel.data:
    public_url = tunnel.data['public_url']
else:
    # Fallback : extraire depuis la représentation string
    import re
    url_match = re.search(r'https://[^"]+\.ngrok[^"]+', str(tunnel))
    public_url = url_match.group(0) if url_match else str(tunnel)

print("\n" + "=" * 80)
print(f"🚀 API PUBLIQUE : {public_url}")
print(f"📡 Health : {public_url}/health")
print(f"💬 Init : {public_url}/init")
print(f"💬 Chat : POST {public_url}/chat")
print("=" * 80)
print(f"\n✅ Serveur lancé sur le port {PORT} !")
print(f"📋 Copie cette URL dans index_spinoza.html : {public_url}")


ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8599): [errno 98] address already in use



🚀 API PUBLIQUE : https://nonremunerative-rory-unbreakably.ngrok-free.dev
📡 Health : https://nonremunerative-rory-unbreakably.ngrok-free.dev/health
💬 Init : https://nonremunerative-rory-unbreakably.ngrok-free.dev/init
💬 Chat : POST https://nonremunerative-rory-unbreakably.ngrok-free.dev/chat

✅ Serveur lancé sur le port 8599 !
📋 Copie cette URL dans index_spinoza.html : https://nonremunerative-rory-unbreakably.ngrok-free.dev


In [11]:
# =============================================================================
# ⚖️ PROMPT D'ÉVALUATION (Calibré pour Mistral 7B)
# =============================================================================

PROMPT_EVALUATION = """Tu évalues un dialogue entre Spinoza et un élève.

DIALOGUE :
{dialogue}

GRILLE D'ÉVALUATION (0-10 pour chaque critère) :

1️⃣ COMPRÉHENSION :
   • 9-10 : Reformule correctement les idées, pose des questions pertinentes, fait des liens
   • 6-8 : Comprend mais demande des clarifications, fait quelques erreurs
   • 3-5 : Comprend partiellement, confusions fréquentes
   • 0-2 : Ne comprend pas, dit "je comprends pas", refuse le dialogue

2️⃣ COOPÉRATION :
   • 9-10 : Répond aux questions, fait avancer le dialogue, s'engage activement
   • 6-8 : Coopère mais avec quelques résistances (ex: "En voilà un pâté !")
   • 3-5 : Résiste souvent, dialogue difficile
   • 0-2 : Refuse de coopérer, dit "j'ai autre chose à faire", part

3️⃣ PROGRESSION :
   • 9-10 : Pensée évolue clairement, conclusions à la fin (ex: "Je comprends maintenant!")
   • 6-8 : Progresse lentement, quelques retours en arrière
   • 3-5 : Peu de progression, tourne en rond
   • 0-2 : Aucune évolution, répète les mêmes incompréhensions

⚠️ PRINCIPE BIENVEILLANT : Un élève qui questionne intelligemment, conteste avec arguments, ou relève des incohérences mérite une BONNE note (7-9).

EXEMPLES CONCRETS :
• "J'en ai rien à faire. Ciao." → comp=1, coop=1, prog=0
• "Je comprends pas" (répété) → comp=3, coop=5, prog=2
• "Ah oui ! c'est plus précis !" (progression visible) → comp=7, coop=8, prog=8
• "Je comprends maintenant. Merci !" → comp=9, coop=10, prog=10

Réponds UNIQUEMENT avec ce JSON (aucun texte avant/après) :

{{
 "comprehension": X,
 "cooperation": Y,
 "progression": Z,
 "total": X+Y+Z
}}"""

In [12]:
# =============================================================================
# 🧪 CELLULE TEST DIRECT : Stopping Criteria (sans API)
# =============================================================================
# Test local dans Colab - Pas besoin du serveur lancé
# Copier-coller après avoir exécuté la cellule CELLULE_INTEGRATION_STOPPING_CRITERIA
# =============================================================================

import time

print("=" * 80)
print("🧪 TEST DIRECT : Baseline vs Stopping Criteria")
print("=" * 80)
print()

# Question de test
test_question = "La liberté est-elle une illusion ?"

print(f"📝 Question : \"{test_question}\"")
print()

# Réinitialiser historique pour test propre
conversation_history_backup = conversation_history.copy()
conversation_history = []

# =============================================================================
# TEST 1 : BASELINE (SANS stopping criteria)
# =============================================================================

print("1️⃣ TEST BASELINE (SANS stopping criteria)")
print("-" * 80)
print()

start_baseline = time.time()

try:
    # Générer SANS stopping criteria
    reply_baseline = spinoza_repond_avec_stopping(
        message=test_question,
        use_stopping_criteria=False  # ← SANS stopping
    )
    latency_baseline = time.time() - start_baseline

    print(f"✅ Réponse générée en {latency_baseline:.2f}s")
    print()
    print(f"📝 Réponse :")
    print(f"   {reply_baseline}")
    print()
    print(f"📊 Stats :")
    print(f"   - Latence : {latency_baseline:.2f}s")
    print(f"   - Tokens (approx) : {len(reply_baseline.split())}")
    print()

except Exception as e:
    print(f"❌ Erreur : {e}")
    latency_baseline = None
    reply_baseline = None

print()

# =============================================================================
# TEST 2 : OPTIMIZED (AVEC stopping criteria)
# =============================================================================

print("2️⃣ TEST OPTIMIZED (AVEC stopping criteria)")
print("-" * 80)
print()

# Réinitialiser historique pour comparaison équitable
conversation_history = []

start_optimized = time.time()

try:
    # Générer AVEC stopping criteria
    reply_optimized = spinoza_repond_avec_stopping(
        message=test_question,
        use_stopping_criteria=True  # ← AVEC stopping
    )
    latency_optimized = time.time() - start_optimized

    print(f"✅ Réponse générée en {latency_optimized:.2f}s")
    print()
    print(f"📝 Réponse :")
    print(f"   {reply_optimized}")
    print()
    print(f"📊 Stats :")
    print(f"   - Latence : {latency_optimized:.2f}s")
    print(f"   - Tokens (approx) : {len(reply_optimized.split())}")
    print()

except Exception as e:
    print(f"❌ Erreur : {e}")
    latency_optimized = None
    reply_optimized = None

print()

# =============================================================================
# COMPARAISON
# =============================================================================

if latency_baseline and latency_optimized:
    print("=" * 80)
    print("📊 COMPARAISON")
    print("=" * 80)
    print()

    # Calculer gain
    gain_abs = latency_baseline - latency_optimized
    gain_pct = (gain_abs / latency_baseline) * 100

    print(f"   Baseline (SANS stopping)   : {latency_baseline:.2f}s")
    print(f"   Optimized (AVEC stopping)  : {latency_optimized:.2f}s")
    print()
    print(f"   🎯 Gain absolu  : {gain_abs:.2f}s")
    print(f"   🎯 Gain relatif : {gain_pct:+.1f}%")
    print()

    # Évaluation du gain
    if gain_pct > 25:
        print("   ✅ EXCELLENT : Gain > 25% (conforme aux tests)")
    elif gain_pct > 15:
        print("   ✅ BON : Gain > 15%")
    elif gain_pct > 5:
        print("   ⚠️  MOYEN : Gain entre 5-15% (acceptable)")
    elif gain_pct > 0:
        print("   ⚠️  FAIBLE : Gain < 5% (vérifier configuration)")
    else:
        print("   ❌ NÉGATIF : Optimized plus lent (problème !)")

    print()
    print("=" * 80)

    # Vérifier qualité réponse (complétude)
    print()
    print("🔍 VÉRIFICATION QUALITÉ RÉPONSES")
    print("=" * 80)
    print()

    import re

    # Compter phrases complètes (finissant par ponctuation)
    def count_complete_sentences(text):
        # Chercher phrases finissant par . ! ?
        sentences = re.findall(r'[^.!?]+[.!?]+', text)
        return len(sentences)

    baseline_sentences = count_complete_sentences(reply_baseline)
    optimized_sentences = count_complete_sentences(reply_optimized)

    print(f"   Baseline sentences  : {baseline_sentences}")
    print(f"   Optimized sentences : {optimized_sentences}")
    print()

    # Vérifier que la réponse optimized est complète
    # (doit finir par ponctuation)
    if reply_optimized.strip().endswith(('.', '!', '?')):
        print("   ✅ Réponse optimized COMPLÈTE (finit par ponctuation)")
    else:
        print("   ⚠️  Réponse optimized INCOMPLÈTE (ne finit pas par ponctuation)")

    print()

else:
    print("⚠️  Impossible de comparer (une des générations a échoué)")
    print()

# Restaurer historique original
conversation_history = conversation_history_backup

print("=" * 80)
print("✅ TEST TERMINÉ")
print("=" * 80)
print()
print("🔧 Si le gain est < 5%, vérifier :")
print("   1. VRAM disponible (nvidia-smi)")
print("   2. Aucun autre processus lourd en cours")
print("   3. Stopping criteria bien actif (vérifier code)")
print()
print("🎯 Gain attendu : +25-35% (testé : 8.10s → 5.68s = +30%)")


🧪 TEST DIRECT : Baseline vs Stopping Criteria

📝 Question : "La liberté est-elle une illusion ?"

1️⃣ TEST BASELINE (SANS stopping criteria)
--------------------------------------------------------------------------------

✅ Réponse générée en 16.09s

📝 Réponse :
   Pour moi la liberté EST cette connaissance. Ce n'est pas une illusion, c'est la véritable chose. Mais ça peut sembler illusoire car tu as confondu 'liberté' et 'servitude'.

📊 Stats :
   - Latence : 16.09s
   - Tokens (approx) : 28


2️⃣ TEST OPTIMIZED (AVEC stopping criteria)
--------------------------------------------------------------------------------

✅ Réponse générée en 13.29s

📝 Réponse :
   Ici, la liberté EST illusoire. Car elle vient de l'ignorance. Ignorer les causes de mes désirs me fait croire que je choisi librement.

📊 Stats :
   - Latence : 13.29s
   - Tokens (approx) : 23


📊 COMPARAISON

   Baseline (SANS stopping)   : 16.09s
   Optimized (AVEC stopping)  : 13.29s

   🎯 Gain absolu  : 2.81s
   🎯 Gain rel